# TextBlob을 이용한 자연어처리 - 긍,부정 문장
- Textblob는 텍스트 정보를 처리하는 파이썬 라이브러리
- 품사 태깅, 명사구 추출, 감정 분석 및 분류, 번역 등과 같은 일반적인 자연 언어 처리(Natural Language Processing, NLP)를 위한 API를 제공


# Language Detection

In [2]:
from textblob import TextBlob as tb
import nltk
import pandas as pd

In [3]:
txt1=tb('맥주가 좋았습니다.')
txt2=tb('Beer was good.')
print('txt1 : ', txt1.detect_language())
print('txt2 : ', txt2.detect_language())

txt1 :  ko
txt2 :  en


# Translation

In [5]:
txt=tb('맥주가 좋았습니다.')
txt.translate(to='en')

TextBlob("Beer was good.")

In [6]:
txt=tb('Beer was good.')
txt.translate(to='ko')

TextBlob("맥주는 좋았습니다.")

# 영문 텍스트 감정 분석
- 간단한 문장을 학습하여 긍정(pos)과 부정(neg)을 분류하는 단순한 분석기를 구현
- NaiveBayesClassifier 참조: https://excelsior-cjh.tistory.com/45 / https://wikidocs.net/22892

In [7]:
from textblob.classifiers import NaiveBayesClassifier

train = [('I love this sandwich.', 'pos'),
         ('This is an amazing place!', 'pos'),
         ('I feel very good about these beers.', 'pos'),
         ('This is my best work.', 'pos'),
         ("What an awesome view", 'pos'),
         ('I do not like this restaurant', 'neg'),
         ('I am tired of this stuff.', 'neg'),
         ("I can't deal with this", 'neg'),
         ('He is my sworn enemy!', 'neg'),
         ('My boss is horrible.', 'neg')
        ]

test = [('The beer was good.', 'pos'),
        ('I do not enjoy my job', 'neg'),
        ("I ain't feeling dandy today.", 'neg'),
        ("I feel amazing!", 'pos'),
        ('Gary is a friend of mine.', 'pos'),
        ("I can't believe I'm doing this.", 'neg')
       ]

In [13]:
print(len(train), train[0])
print(len(test), test[0])

10 ('I love this sandwich.', 'pos')
6 ('The beer was good.', 'pos')


- train

In [14]:
pos_neg = NaiveBayesClassifier(train)

- test

In [15]:
print(pos_neg.classify("Their burgers are amazing"))  # "pos"
print(pos_neg.classify("I don't like their pizza."))  # "neg"

pos
neg


In [20]:
for i, t in enumerate(test):
    print(f'{i+1}번 문장: 정답 >>> {t[1]}, 예측 >>> {pos_neg.classify(t[0])}')

1번 문장: 정답 >>> pos, 예측 >>> pos
2번 문장: 정답 >>> neg, 예측 >>> neg
3번 문장: 정답 >>> neg, 예측 >>> neg
4번 문장: 정답 >>> pos, 예측 >>> pos
5번 문장: 정답 >>> pos, 예측 >>> neg
6번 문장: 정답 >>> neg, 예측 >>> neg


In [19]:
# TextBlob 객체를 사용하여 여러 문장으로 구성된 텍스트도 분류

tag2 = tb("The beer was amazing. "
         "But the hangover was horrible. My boss was not happy.",
         classifier=pos_neg)
tag2.classify()  # 전체 문장을 한 문장으로 예측한 결과

'neg'

In [25]:
for sentence in tag2.sentences:
    print(sentence)
    print(sentence.classify())
    print('-----------------')

The beer was amazing.
pos
-----------------
But the hangover was horrible.
neg
-----------------
My boss was not happy.
neg
-----------------


In [27]:
# test 문장을 통해 정확도 확인

pos_neg.accuracy(test)

0.8333333333333334

In [31]:
# 분류기준

pos_neg.show_informative_features(5)

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0


# 한글 텍스트 감정 분석

In [32]:
train = [('나는 이 샌드위치를 정말 좋아해.', '긍정'),
         ('정말 멋진 곳이에요!', '긍정'),
         ('나는 이 맥주들이 아주 좋다고 생각해요.', '긍정'),
         ('이것은 나의 최고의 작품입니다.', '긍정'),
         ("정말 멋진 광경이다", "긍정"),
         ('난 이 식당 싫어', '부정'),
         ('난 이게 지겨워.', '부정'),
         ("이 문제는 처리할 수 없습니다.", "부정"),
         ('그는 나의 불구대천의 원수이다.', '부정'),
         ('내 상사는 끔찍해.', '부정'),
         ('나는 내 꿈을 믿는다', '긍정'),
         ('나는 매일 최선을 다하고 있다', '긍정'),
         ('나는 있는 그대로의 나를 사랑한다', '긍정'),
         ('나는 내 삶을 100% 책임진다', '긍정'),
         ('가장 좋은 일은 아직 생기지 않았다', '긍정'),
         ('나는 매일 나의 삶에 감사한다', '긍정'),
         ('새로나온 휴대폰은 배터리 교체가 되지 않아 불편하다', '부정'),
         ('이번에 나온 영화 너무 재밌다. 주말에 또 보고 싶다.', '긍정'),
         ('나의 아버지는 이해가 안된다', '부정'),
         ('나는 어머니와 있을 때 퉁명해진다', '부정'),
         ('나는 어머니와 있을 때 불편할 때가 있다.', '부정'),
         ('치킨집에 선결제로 주문했는데 돈은 결제가 되었는데 치킨집에선 주문이 안들어왔다고합니다.','부정'),
         ('지금 현재 네이버 실시간 검색어에 떴고, 여전히 고객센터는 전화연결이 되지않습니다. ','부정'),
         ('이럴거면 요기요처럼 아예 접속도 안되게하던가 결제를 막았어야하는데','부정'),
         ('결제해서 가게에 돈 준 것도 아니고 본인들이 가져가놓고 전화도 안받으면 돈은 어떻게 돌려받습니까?','부정'),
         ('너무 유용하고 편리하네요.. ','긍정'),
         ('처음 구입한다고 쿠폰 받고 이용하니깐 저렴하게 먹을 수 있었고 배달시간도 만족스러워서 좋았습니다.','긍정'),
         ('배달의 민족 강추 합니다. ','긍정'),
         ('자주 이용할께요ㅡㅡㅡ강추 강추!!!','긍정')
    
        ]

test = [('맥주가 좋았습니다.', '긍정'),
        ('난 내 일을 즐기지 않는다', '부정'),
        ('오늘은 기분이 안 좋아요.', '부정'),
        ('놀라워요!', '긍정'),
        ('네드는 나의 친구입니다.', '긍정'),
        ('제가 이렇게 하고 있다니 믿을 수가 없어요.', '부정')
       ]

In [33]:
print(len(train), train[0])
print(len(test), test[0])

29 ('나는 이 샌드위치를 정말 좋아해.', '긍정')
6 ('맥주가 좋았습니다.', '긍정')


# 형태소 분석 없이 학습

In [34]:
pos_neg = NaiveBayesClassifier(train)  # 데이터 학습
pos_neg.show_informative_features()    # 분류기준 확인

Most Informative Features
            contains(나는) = True               긍정 : 부정     =      2.5 : 1.0
            contains(나는) = False              부정 : 긍정     =      1.5 : 1.0
             contains(내) = True               긍정 : 부정     =      1.4 : 1.0
            contains(나의) = True               부정 : 긍정     =      1.2 : 1.0
             contains(이) = True               부정 : 긍정     =      1.2 : 1.0
            contains(정말) = False              부정 : 긍정     =      1.2 : 1.0
             contains(수) = True               부정 : 긍정     =      1.2 : 1.0
            contains(있다) = True               부정 : 긍정     =      1.2 : 1.0
             contains(난) = False              긍정 : 부정     =      1.2 : 1.0
            contains(돈은) = False              긍정 : 부정     =      1.2 : 1.0


In [36]:
pos_neg.accuracy(test)

0.5

In [38]:
for i, t in enumerate(test):
    print(f'{i+1}번 문장: 정답: {t[1]}, 예측: {pos_neg.classify(t[0])}')

1번 문장: 정답: 긍정, 예측: 긍정
2번 문장: 정답: 부정, 예측: 긍정
3번 문장: 정답: 부정, 예측: 긍정
4번 문장: 정답: 긍정, 예측: 긍정
5번 문장: 정답: 긍정, 예측: 긍정
6번 문장: 정답: 부정, 예측: 긍정


# 형태소 분석 후 데이터 학습

In [42]:
# train 데이터의 텍스트를 형태소 분석하여 train_data를 생성
from konlpy.tag import Okt
okt=Okt()

# 한글 단어 뒤에 태그를 붙여줘야 분류기가 정확하게 분류할 수 있음
train_data = [(['/'.join(token) for token in okt.pos(sentence)], result) for [sentence, result] in train]

print(train_data[:5])

[(['나/Noun', '는/Josa', '이/Noun', '샌드위치/Noun', '를/Josa', '정말/Noun', '좋아해/Adjective', './Punctuation'], '긍정'), (['정말/Noun', '멋진/Adjective', '곳/Noun', '이에요/Josa', '!/Punctuation'], '긍정'), (['나/Noun', '는/Josa', '이/Noun', '맥주/Noun', '들/Suffix', '이/Josa', '아주/Noun', '좋다고/Adjective', '생각/Noun', '해요/Verb', './Punctuation'], '긍정'), (['이/Determiner', '것/Noun', '은/Josa', '나/Noun', '의/Josa', '최고/Noun', '의/Josa', '작품/Noun', '입니다/Adjective', './Punctuation'], '긍정'), (['정말/Noun', '멋진/Adjective', '광경/Noun', '이다/Josa'], '긍정')]


In [49]:
pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

In [50]:
test_data = [(['/'.join(token) for token in okt.pos(sentence)], result) for [sentence, result] in test]

pos_neg.accuracy(test_data)

0.5

In [75]:
for i, t in enumerate(test_data):
    print(f'{i+1}번 문장: 정답: {t[1]}, 예측: {pos_neg.classify(t[0])}')

1번 문장: 정답: 긍정, 예측: 긍정
2번 문장: 정답: 부정, 예측: 긍정
3번 문장: 정답: 부정, 예측: 긍정
4번 문장: 정답: 긍정, 예측: 긍정
5번 문장: 정답: 긍정, 예측: 긍정
6번 문장: 정답: 부정, 예측: 긍정


In [51]:
pos_neg.show_informative_features()

Most Informative Features
        contains(이/Josa) = True               부정 : 긍정     =      2.8 : 1.0
        contains(도/Josa) = True               부정 : 긍정     =      2.0 : 1.0
        contains(에/Josa) = True               부정 : 긍정     =      1.7 : 1.0
        contains(은/Josa) = True               부정 : 긍정     =      1.7 : 1.0
        contains(이/Noun) = True               부정 : 긍정     =      1.7 : 1.0
        contains(나/Noun) = True               긍정 : 부정     =      1.6 : 1.0
        contains(의/Josa) = True               긍정 : 부정     =      1.5 : 1.0
        contains(가/Josa) = False              긍정 : 부정     =      1.4 : 1.0
        contains(내/Noun) = True               긍정 : 부정     =      1.4 : 1.0
        contains(나/Noun) = False              부정 : 긍정     =      1.4 : 1.0


# 배민 어플 후기 감성 분석

In [95]:
import pandas as pd

df = pd.read_csv('./py_data/sentimental/배민어플후기.csv', encoding='euc-kr')

C:\Users\dojun\anaconda3\envs\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,153,154,155,156,157) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [96]:
df = df[['Reviews', 'pos_neg']][:50]
df.tail()

,Reviews,pos_neg
45,쿠폰사논게 있어서 쓰려고 주문하기 하니 계속 오류네요ㅜㅜ 주문하기로 넘어가질 않아요ㅜㅜ,부정
46,편하고 좋아요 근데 쿠폰이벤때 업체에서 주문취소하고 전화안받고 개떡같은 업주들 몇있음,부정
47,배민 쿠폰 너무 좋아요~덕분에 자주 시켜 먹어서 지출도 늘었지만 편하고 좋아요,긍정
48,쓰리치킨 구성도좋고 한번에 세가지를 즐길수있어 넘 좋아요~~ 배달임에도 불구하고 갓...,긍정
49,배민 카테고리별 정리도 잘되있고 맛집선정해서 따로 나뉘서있어서 이용이 편하고 좋아요,긍정


In [102]:
tmp_train = df[:40]
tmp_test = df[40:]

In [103]:
print(len(tmp_train))
print(len(tmp_test))

40
10


In [116]:
train = []
for t in tmp_train.iterrows():
    train.append((t[1][0], t[1][1]))

In [117]:
test = []
for t in tmp_test.iterrows():
    test.append((t[1][0], t[1][1]))

In [118]:
pos_neg = NaiveBayesClassifier(train)  # 데이터 학습
pos_neg.show_informative_features()    # 분류기준 확인

Most Informative Features
           contains(앞으로) = True               부정 : 긍정     =      4.7 : 1.0
            contains(많이) = True               부정 : 긍정     =      3.3 : 1.0
           contains(배달이) = True               부정 : 긍정     =      3.3 : 1.0
           contains(그런데) = True               부정 : 긍정     =      2.0 : 1.0
             contains(다) = True               부정 : 긍정     =      2.0 : 1.0
            contains(믿고) = True               부정 : 긍정     =      2.0 : 1.0
            contains(배민) = True               부정 : 긍정     =      2.0 : 1.0
             contains(앱) = True               부정 : 긍정     =      2.0 : 1.0
            contains(와서) = True               부정 : 긍정     =      2.0 : 1.0
           contains(있어서) = True               부정 : 긍정     =      2.0 : 1.0


In [119]:
pos_neg.accuracy(test)

0.6

## 배민 리뷰 감정 분석 - 형태소 분석

In [121]:
# train 데이터의 텍스트를 형태소 분석하여 train_data를 생성
from konlpy.tag import Okt
okt=Okt()

# 한글 단어 뒤에 태그를 붙여줘야 분류기가 정확하게 분류할 수 있음
train_data = [(['/'.join(token) for token in okt.pos(sentence)], result) for [sentence, result] in train]

print(train_data[:2])

[(['어이/Noun', '가/Josa', '없어서/Adjective', '말/Noun', '도안/Noun', '나오네요/Verb', '받지도/Verb', '않은/Verb', '음식/Noun', '을/Josa', '결제/Noun', '하/Suffix', '라는/Josa', '게말/Noun', '이/Josa', '되는건/Verb', '가요/Noun', '?/Punctuation', '출근/Noun', '도/Josa', '해야하는데/Verb', '이/Determiner', '늦/Noun', '은/Josa', '시간/Noun', '에/Josa', '전화/Noun', '오는것도/Verb', '이해/Noun', '를/Josa', '못/Noun', '하겠네요/Verb', '덕분/Noun', '에/Josa', '자다가/Verb', '깼네요/Verb', 'ㅡㅡ/KoreanParticle', '이/Determiner', '제이/Noun', '어플/Noun', '안/VerbPrefix', '씁니다/Verb', 'ㅡㅡ/KoreanParticle', '이따위/Modifier', '로/Noun', '하는거아니라고/Verb', '봅니다/Verb', 'ㅡㅡ/KoreanParticle', '와/Verb', '타부/Noun', '타/Modifier', '말/Noun', '도/Josa', '없이/Adverb', '결제/Noun', '라니/Josa', '../Punctuation', '어이/Noun', '가/Josa', '없네요/Adjective', '../Punctuation', 'ㅡㅡ/KoreanParticle'], '부정'), (['제/Noun', '동생/Noun', '추천/Noun', '으로/Josa', '주문/Noun', '해본건데/Verb', '동생/Noun', '말/Noun', '이/Josa', '틀리지/Verb', '않았어요/Verb', './Punctuation', '브랜드/Noun', '친/Noun', '키보/Noun', '다/Josa', '제/Noun', '입맛/Noun',

In [122]:
pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

In [123]:
test_data = [(['/'.join(token) for token in okt.pos(sentence)], result) for [sentence, result] in test]

pos_neg.accuracy(test_data)

0.6

In [124]:
pos_neg.show_informative_features()

Most Informative Features
        contains(가/Josa) = True               부정 : 긍정     =      6.0 : 1.0
     contains(많이/Adverb) = True               부정 : 긍정     =      4.7 : 1.0
       contains(배민/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(앱/Noun) = True               부정 : 긍정     =      4.7 : 1.0
      contains(합니다/Verb) = True               부정 : 긍정     =      4.7 : 1.0
       contains(혜택/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(은/Josa) = True               부정 : 긍정     =      4.4 : 1.0
        contains(가/Josa) = False              긍정 : 부정     =      3.5 : 1.0
        contains(곳/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(리뷰/Noun) = True               부정 : 긍정     =      3.3 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 조사 제거

In [131]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] != 'Josa')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] != 'Josa')], result) for [sentence, result] in test]

In [132]:
pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

In [133]:
pos_neg.accuracy(test_data)

0.6

In [134]:
pos_neg.show_informative_features()

Most Informative Features
     contains(많이/Adverb) = True               부정 : 긍정     =      4.7 : 1.0
       contains(배민/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(앱/Noun) = True               부정 : 긍정     =      4.7 : 1.0
      contains(합니다/Verb) = True               부정 : 긍정     =      4.7 : 1.0
       contains(혜택/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(곳/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(리뷰/Noun) = True               부정 : 긍정     =      3.3 : 1.0
  contains(못/VerbPrefix) = True               부정 : 긍정     =      3.3 : 1.0
       contains(사람/Noun) = True               부정 : 긍정     =      3.3 : 1.0
      contains(서비스/Noun) = True               부정 : 긍정     =      3.3 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 명사

In [136]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Noun')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Noun')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.6
Most Informative Features
       contains(배민/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(앱/Noun) = True               부정 : 긍정     =      4.7 : 1.0
       contains(혜택/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(곳/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(리뷰/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(사람/Noun) = True               부정 : 긍정     =      3.3 : 1.0
      contains(서비스/Noun) = True               부정 : 긍정     =      3.3 : 1.0
        contains(앞/Noun) = True               부정 : 긍정     =      2.8 : 1.0
        contains(더/Noun) = True               부정 : 긍정     =      2.0 : 1.0
        contains(말/Noun) = True               부정 : 긍정     =      2.0 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 형용사

In [137]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.6
Most Informative Features
 contains(좋은데/Adjective) = True               부정 : 긍정     =      3.3 : 1.0
contains(없었는데/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(있어서/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(편하고/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(맛있는/Adjective) = True               긍정 : 부정     =      1.5 : 1.0
 contains(입니다/Adjective) = False              부정 : 긍정     =      1.3 : 1.0
 contains(좋아요/Adjective) = False              부정 : 긍정     =      1.3 : 1.0
 contains(맛있게/Adjective) = False              부정 : 긍정     =      1.2 : 1.0
contains(맛있어요/Adjective) = False              부정 : 긍정     =      1.2 : 1.0
  contains(좋은/Adjective) = True               부정 : 긍정     =      1.2 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 부사

In [138]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adverb')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adverb')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.5
Most Informative Features
     contains(많이/Adverb) = True               부정 : 긍정     =      4.7 : 1.0
     contains(없이/Adverb) = True               부정 : 긍정     =      2.0 : 1.0
      contains(다/Adverb) = True               부정 : 긍정     =      1.4 : 1.0
     contains(많이/Adverb) = False              긍정 : 부정     =      1.3 : 1.0
   contains(관계없이/Adverb) = False              긍정 : 부정     =      1.1 : 1.0
     contains(너무/Adverb) = True               긍정 : 부정     =      1.1 : 1.0
      contains(다/Adverb) = False              긍정 : 부정     =      1.1 : 1.0
     contains(없이/Adverb) = False              긍정 : 부정     =      1.1 : 1.0
      contains(딱/Adverb) = False              부정 : 긍정     =      1.1 : 1.0
     contains(너무/Adverb) = False              부정 : 긍정     =      1.0 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 형용사, KoreanParticle

In [139]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.6
Most Informative Features
 contains(좋은데/Adjective) = True               부정 : 긍정     =      3.3 : 1.0
contains(ㅋ/KoreanParticle) = True               부정 : 긍정     =      2.0 : 1.0
contains(없었는데/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(있어서/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(편하고/Adjective) = True               부정 : 긍정     =      2.0 : 1.0
 contains(맛있는/Adjective) = True               긍정 : 부정     =      1.5 : 1.0
 contains(입니다/Adjective) = False              부정 : 긍정     =      1.3 : 1.0
 contains(좋아요/Adjective) = False              부정 : 긍정     =      1.3 : 1.0
 contains(맛있게/Adjective) = False              부정 : 긍정     =      1.2 : 1.0
contains(맛있어요/Adjective) = False              부정 : 긍정     =      1.2 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 명사, 형용사, KoreanParticle

In [140]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')|(token[1] == 'Noun')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')|(token[1] == 'Noun')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.6
Most Informative Features
       contains(배민/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(앱/Noun) = True               부정 : 긍정     =      4.7 : 1.0
       contains(혜택/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(곳/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(리뷰/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(사람/Noun) = True               부정 : 긍정     =      3.3 : 1.0
      contains(서비스/Noun) = True               부정 : 긍정     =      3.3 : 1.0
 contains(좋은데/Adjective) = True               부정 : 긍정     =      3.3 : 1.0
        contains(앞/Noun) = True               부정 : 긍정     =      2.8 : 1.0
contains(ㅋ/KoreanParticle) = True               부정 : 긍정     =      2.0 : 1.0


## 배민 리뷰 감정 분석 - 형태소 분석 / 명사, 형용사, 부사, KoreanParticle

In [142]:
train_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')|(token[1] == 'Noun')|(token[1] == 'Adverb')], result) for [sentence, result] in train]
test_data = [(['/'.join(token) for token in okt.pos(sentence) if (token[1] == 'Adjective')|(token[1] == 'KoreanParticle')|(token[1] == 'Noun')|(token[1] == 'Adverb')], result) for [sentence, result] in test]

pos_neg = NaiveBayesClassifier(train_data)  # 데이터 학습

print('accuracy: ', pos_neg.accuracy(test_data))
pos_neg.show_informative_features()

accuracy:  0.6
Most Informative Features
     contains(많이/Adverb) = True               부정 : 긍정     =      4.7 : 1.0
       contains(배민/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(앱/Noun) = True               부정 : 긍정     =      4.7 : 1.0
       contains(혜택/Noun) = True               부정 : 긍정     =      4.7 : 1.0
        contains(곳/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(리뷰/Noun) = True               부정 : 긍정     =      3.3 : 1.0
       contains(사람/Noun) = True               부정 : 긍정     =      3.3 : 1.0
      contains(서비스/Noun) = True               부정 : 긍정     =      3.3 : 1.0
 contains(좋은데/Adjective) = True               부정 : 긍정     =      3.3 : 1.0
        contains(앞/Noun) = True               부정 : 긍정     =      2.8 : 1.0


In [145]:
for i, data in enumerate(test_data):
    print(f'{i+1}번 문장 >>> 정답: {data[1]}, 예측: {pos_neg.classify(data[0])}')

1번 문장 >>> 정답: 긍정, 예측: 긍정
2번 문장 >>> 정답: 부정, 예측: 긍정
3번 문장 >>> 정답: 부정, 예측: 긍정
4번 문장 >>> 정답: 긍정, 예측: 긍정
5번 문장 >>> 정답: 긍정, 예측: 긍정
6번 문장 >>> 정답: 부정, 예측: 긍정
7번 문장 >>> 정답: 부정, 예측: 긍정
8번 문장 >>> 정답: 긍정, 예측: 긍정
9번 문장 >>> 정답: 긍정, 예측: 긍정
10번 문장 >>> 정답: 긍정, 예측: 긍정


In [146]:
for i, data in enumerate(train_data):
    print(f'{i+1}번 문장 >>> 정답: {data[1]}, 예측: {pos_neg.classify(data[0])}')

1번 문장 >>> 정답: 부정, 예측: 부정
2번 문장 >>> 정답: 긍정, 예측: 긍정
3번 문장 >>> 정답: 부정, 예측: 부정
4번 문장 >>> 정답: 부정, 예측: 부정
5번 문장 >>> 정답: 긍정, 예측: 긍정
6번 문장 >>> 정답: 긍정, 예측: 긍정
7번 문장 >>> 정답: 긍정, 예측: 긍정
8번 문장 >>> 정답: 부정, 예측: 부정
9번 문장 >>> 정답: 긍정, 예측: 긍정
10번 문장 >>> 정답: 부정, 예측: 부정
11번 문장 >>> 정답: 긍정, 예측: 긍정
12번 문장 >>> 정답: 긍정, 예측: 긍정
13번 문장 >>> 정답: 긍정, 예측: 긍정
14번 문장 >>> 정답: 긍정, 예측: 긍정
15번 문장 >>> 정답: 긍정, 예측: 긍정
16번 문장 >>> 정답: 긍정, 예측: 긍정
17번 문장 >>> 정답: 긍정, 예측: 긍정
18번 문장 >>> 정답: 긍정, 예측: 긍정
19번 문장 >>> 정답: 긍정, 예측: 긍정
20번 문장 >>> 정답: 긍정, 예측: 긍정
21번 문장 >>> 정답: 부정, 예측: 부정
22번 문장 >>> 정답: 긍정, 예측: 긍정
23번 문장 >>> 정답: 긍정, 예측: 긍정
24번 문장 >>> 정답: 긍정, 예측: 긍정
25번 문장 >>> 정답: 부정, 예측: 부정
26번 문장 >>> 정답: 긍정, 예측: 긍정
27번 문장 >>> 정답: 긍정, 예측: 긍정
28번 문장 >>> 정답: 부정, 예측: 부정
29번 문장 >>> 정답: 긍정, 예측: 긍정
30번 문장 >>> 정답: 긍정, 예측: 긍정
31번 문장 >>> 정답: 긍정, 예측: 긍정
32번 문장 >>> 정답: 긍정, 예측: 긍정
33번 문장 >>> 정답: 부정, 예측: 부정
34번 문장 >>> 정답: 부정, 예측: 부정
35번 문장 >>> 정답: 부정, 예측: 부정
36번 문장 >>> 정답: 긍정, 예측: 긍정
37번 문장 >>> 정답: 부정, 예측: 부정
38번 문장 >>> 정답: 부정, 예측: 부정
39번 문장 >>> 정답: 긍정, 예측